<a href="https://colab.research.google.com/github/amirhossini/Pyomo-Educational-Notebooks/blob/main/Pyomo2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pyomo Examples

__Notebook:__ Non-Linear Prpogramming (NLP) 

__Questions:__ amir.hossini@queensu.ca

_Libraries_

In [1]:
! pip install pyomo                 # software package for formulating optimization problems
! apt-get install -y -qq glpk-utils # software package with solver for large scale LP and MILP problems
import pyomo.environ as pyomo

! wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
! unzip -o -q ipopt-linux64

     |████████████████████████████████| 9.6 MB 14.0 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155455 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up

### Example 1 - Regreassion

In [2]:
#### Model definition
model_linear = pyomo.ConcreteModel()
model_nonlinear = pyomo.ConcreteModel()

#### Set declaration
model_linear.m = pyomo.RangeSet(6)
model_nonlinear.m = pyomo.RangeSet(6)

#### Variable definition
model_linear.a1 = pyomo.Var(domain=pyomo.Reals)
model_linear.a2 = pyomo.Var(domain=pyomo.Reals)

model_nonlinear.b1 = pyomo.Var(domain=pyomo.Reals, initialize = 500)
model_nonlinear.b2 = pyomo.Var(domain=pyomo.Reals, initialize = -150)
model_nonlinear.b3 = pyomo.Var(domain=pyomo.Reals, initialize = -0.2, bounds=(-5,5))

#### Parameter declaration 
model_linear.datapoints_y = pyomo.Param(model_linear.m, 
                                        initialize = {1:127,2:151,3:379,
                                                      4:421,5:460,6:426})
model_linear.datapoints_x = pyomo.Param(model_linear.m, 
                                        initialize = {1:-5,2:-3,3:-1,
                                                      4:5,5:3,6:1})

model_nonlinear.datapoints_y = pyomo.Param(model_nonlinear.m, 
                                        initialize = {1:127,2:151,3:379,
                                                      4:421,5:460,6:426})
model_nonlinear.datapoints_x = pyomo.Param(model_nonlinear.m, 
                                        initialize = {1:-5,2:-3,3:-1,
                                                      4:5,5:3,6:1})

#### Objective functions
model_linear.obj = pyomo.Objective(expr = sum((model_linear.datapoints_y[m]-
                                               (model_linear.a1+model_linear.a2*model_linear.datapoints_x[m]))**2 
                                              for m in model_linear.m),sense = pyomo.minimize)

model_nonlinear.obj = pyomo.Objective(expr = sum((model_nonlinear.datapoints_y[m]-
                                               (model_nonlinear.b1+model_nonlinear.b2*pyomo.exp(model_nonlinear.b3*model_nonlinear.datapoints_x[m])))**2 
                                              for m in model_nonlinear.m),sense = pyomo.minimize)

#### Solver options
results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_linear)

results.write()
print("\n RESULTS \n")
print("Squared deviation for linear regression model = ", model_linear.obj())
print('Coefficient 1 for linear regression (a1) = ', model_linear.a1())
print('Coefficient 2 for linear regression (a2) = ', model_linear.a2())

results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_nonlinear)

results.write()
print("\n RESULTS \n")
print("Squared deviation for nonlinear regression model = ", model_nonlinear.obj())
print('Coefficient 1 for linear regression (b1) = ', model_nonlinear.b1())
print('Coefficient 2 for linear regression (b2) = ', model_nonlinear.b2())
print('Coefficient 2 for linear regression (b3) = ', model_nonlinear.b3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.018710851669311523
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 RESU

### Example 2 - Geometirc Optimization

In [3]:
#### Model definition
model_circle = pyomo.ConcreteModel()

#### Set defintion
model_circle.i = pyomo.RangeSet(5)
model_circle.j = pyomo.Set(initialize = model_circle.i) # Creates an alias of set i

#### Variable declaration
model_circle.r = pyomo.Var(domain = pyomo.NonNegativeReals, bounds = (0.05,0.4))

#### Function to initialize coordinate variables
def init(model_circle,l):
  return -0.2 + l * 0.1

model_circle.x = pyomo.Var(model_circle.i,domain=pyomo.Reals,bounds=(-1,1),initialize = init)
model_circle.y = pyomo.Var(model_circle.i,domain=pyomo.Reals,bounds=(-1,1),initialize = init)

#### Constrain definition
def rule4(model_circle,i):
  return (1-model_circle.r)**2 >= (model_circle.x[i]**2)+(model_circle.y[i]**2)
model_circle.eq4 = pyomo.Constraint(model_circle.i,rule=rule4,doc='Containment constraint')

def rule5(model_circle,i,j):
  if i<j:
    return (model_circle.x[i]-model_circle.x[j])**2 + (model_circle.y[i]-model_circle.y[j])**2 >= 4*(model_circle.r**2)
  return pyomo.Constraint.Skip

model_circle.eq5 = pyomo.Constraint(model_circle.i,model_circle.j, rule = rule5, doc = 'No overlap constraint')

#### Objective function
model_circle.obj = pyomo.Objective(expr = model_circle.r, sense = pyomo.maximize)

#### Solve statement
results = pyomo.SolverFactory('ipopt', executable = '/content/ipopt').solve(model_circle)

#### Printing results
results.write()
print('\n RESULTS \n')
print('Radius of identically size circles = ', model_circle.r(),'\n')
for i in model_circle.i:
  print("Coordinate of circle ",i, "=> (", model_circle.x[i](),",",model_circle.y[i](),") \n")

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 11
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.07117772102355957
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

 RES